In [33]:
# step 1. Complete Decomposition Using CEEMDAN

In [1]:
import pandas as pd
from PyEMD import CEEMDAN

# Load the dataset from an Excel file
data = pd.read_excel('./Datasets/Original Data.xlsx')

In [37]:
# Filter the dataset to include only the training data based on the 'time_idx'
train_data = data[data['time_idx'] <= 712]
wti_train = train_data['WTI']
brent_train = train_data['Brent']

# Initialize the CEEMDAN object with a specified number of trials for robust decomposition
ceemdan = CEEMDAN()
ceemdan.trials = 100

# Function to decompose a time series using the CEEMDAN method
def decompose_data(series):
    imfs = ceemdan(series)
     # Calculate residuals by subtracting the sum of the first two IMFs from the original series if more than two IMFs are present
    residual = series - sum(imfs[:2]) if len(imfs) > 2 else series - sum(imfs)
    return imfs[:2], residual

# Decompose the WTI and Brent training datasets
wti_imfs, wti_residual = decompose_data(wti_train.values)
brent_imfs, brent_residual = decompose_data(brent_train.values)

# Append decomposed components to the original dataset and adjust by adding 10 for normalization
# The addition of 10 to the first two IMFs (Intrinsic Mode Functions) is a preprocessing step to normalize their scale.
# Since IMFs often oscillate around zero, this shift facilitates better learning and training stability for subsequent modeling.
# This normalization is reversed in the final analysis to maintain the original data scale.
for i, imf in enumerate(wti_imfs):
    data[f'WTI_IMF{i+1}'] = pd.Series(imf + 10, index=train_data.index) # Adjust WTI IMFs
data['WTI_Res'] = pd.Series(wti_residual, index=train_data.index) # Append WTI residual

for i, imf in enumerate(brent_imfs):
    data[f'Brent_IMF{i+1}'] = pd.Series(imf + 10, index=train_data.index)
data['Brent_Res'] = pd.Series(brent_residual, index=train_data.index)

# Save the enriched dataset back to the Excel file
data.to_excel('./Datasets/Original Data.xlsx', index=False)

In [14]:
# step 2 Extended Window Application

In [50]:
data = pd.read_excel('./Datasets/CEEMDAN Decomposed Data.xlsx')

# Filter data for training based on the 'time_idx'
train_data = data[data['time_idx'] <= 712]
N1 = 30  # Main window size for feature extraction
N2 = 5   # Extension window size

mapping_pairs = []
for t in range(1, len(train_data) - N1 - N2 + 1):
    W_t = train_data.iloc[t-1:t+N1-1]  # Extract the main window data for past N1 days
    D_k_t_plus_N2 = train_data.iloc[t+N2-1:t+N1+N2-1]  # Extract the extended window data for target IMF components at N2 days ahead

    W_t_data = W_t[['WTI', 'time_idx']]
    D_k_data = D_k_t_plus_N2[['WTI_IMF1', 'WTI_IMF2', 'WTI_Res']]
    
    # Collect the pair of feature matrix and target matrix for training
    mapping_pairs.append((W_t_data, D_k_data))

In [51]:
# step 3 Training the Enhancing Mapping Neural Network (EMNN)

In [52]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a function to build a multi-layer neural network model for regression
def build_mnn_model(input_dim):
    # Construct a fully connected neural network with four hidden layers and one output layer
    model = Sequential([
        Dense(1024, input_dim=input_dim, activation='relu'),
        Dense(1024, activation='relu'),
        Dense(1024, activation='relu'),
        Dense(1024, activation='relu'),
        Dense(1)   
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse') # Compile model with MSE loss and Adam optimizer
    return model

# Extract features and targets for training from mapping pairs
train_features = np.array([pair[0].values.flatten() for pair in mapping_pairs])
train_targets_imf1 = np.array([pair[1]['WTI_IMF1'].values[-1] for pair in mapping_pairs]).reshape(-1, 1)
train_targets_imf2 = np.array([pair[1]['WTI_IMF2'].values[-1] for pair in mapping_pairs]).reshape(-1, 1)
train_targets_residual = np.array([pair[1]['WTI_Res'].values[-1] for pair in mapping_pairs]).reshape(-1, 1)

# Build models for each IMF and the residual
input_dim = W_t_data.shape[1] * W_t_data.shape[0]  
imf1_model = build_mnn_model(input_dim)
imf2_model = build_mnn_model(input_dim)
residual_model = build_mnn_model(input_dim)

# Configure callbacks to save the best model based on the lowest loss
checkpoint_imf1 = ModelCheckpoint('./EEMD_saved_models/WTI_imf1_best_model.h5', monitor='loss', save_best_only=True, verbose=1)
checkpoint_imf2 = ModelCheckpoint('./EEMD_saved_models/WTI_imf2_best_model.h5', monitor='loss', save_best_only=True, verbose=1)
checkpoint_residual = ModelCheckpoint('./EEMD_saved_models/WTI_residual_best_model.h5', monitor='loss', save_best_only=True, verbose=1)

# Conditionally train models if all data arrays match in length
if (len(train_features) == len(train_targets_imf1) == len(train_targets_imf2) == len(train_targets_residual)):
    imf1_model.fit(train_features, train_targets_imf1, epochs=50, batch_size=32, verbose=1, callbacks=[checkpoint_imf1])
    imf2_model.fit(train_features, train_targets_imf2, epochs=50, batch_size=32, verbose=1, callbacks=[checkpoint_imf2])
    residual_model.fit(train_features, train_targets_residual, epochs=50, batch_size=32, verbose=1, callbacks=[checkpoint_residual])
else:
    print("Error: Data cardinality mismatch!")

Features size: (677, 60)
Target IMF1 size: (677, 1)
Target IMF2 size: (677, 1)
Target Residual size: (677, 1)
Epoch 1/50
21/22 [===========================>..] - ETA: 0s - loss: 39613.5195
Epoch 1: loss improved from inf to 39321.37891, saving model to ./EEMD_saved_models\WTI_imf1_best_model.h5
22/22 [==============================] - 2s 35ms/step - loss: 39321.3789
Epoch 2/50
21/22 [===========================>..] - ETA: 0s - loss: 24.8974
Epoch 2: loss improved from 39321.37891 to 24.76466, saving model to ./EEMD_saved_models\WTI_imf1_best_model.h5
22/22 [==============================] - 1s 34ms/step - loss: 24.7647
Epoch 3/50
22/22 [==============================] - ETA: 0s - loss: 3.0262
Epoch 3: loss improved from 24.76466 to 3.02620, saving model to ./EEMD_saved_models\WTI_imf1_best_model.h5
22/22 [==============================] - 1s 37ms/step - loss: 3.0262
Epoch 4/50
21/22 [===========================>..] - ETA: 0s - loss: 2.6288
Epoch 4: loss improved from 3.02620 to 2.65073

Epoch 42/50
21/22 [===========================>..] - ETA: 0s - loss: 2.7903
Epoch 42: loss did not improve from 2.38023
22/22 [==============================] - 1s 29ms/step - loss: 2.8025
Epoch 43/50
20/22 [==========================>...] - ETA: 0s - loss: 3.3608
Epoch 43: loss did not improve from 2.38023
22/22 [==============================] - 1s 30ms/step - loss: 3.3946
Epoch 44/50
22/22 [==============================] - ETA: 0s - loss: 2.5894
Epoch 44: loss did not improve from 2.38023
22/22 [==============================] - 1s 28ms/step - loss: 2.5894
Epoch 45/50
21/22 [===========================>..] - ETA: 0s - loss: 4.2081
Epoch 45: loss did not improve from 2.38023
22/22 [==============================] - 1s 29ms/step - loss: 4.1896
Epoch 46/50
21/22 [===========================>..] - ETA: 0s - loss: 2.4323
Epoch 46: loss did not improve from 2.38023
22/22 [==============================] - 1s 29ms/step - loss: 2.4416
Epoch 47/50
21/22 [===========================>..] - ET

Epoch 33/50
21/22 [===========================>..] - ETA: 0s - loss: 4.3983
Epoch 33: loss did not improve from 4.40191
22/22 [==============================] - 1s 29ms/step - loss: 4.5383
Epoch 34/50
21/22 [===========================>..] - ETA: 0s - loss: 5.0100
Epoch 34: loss did not improve from 4.40191
22/22 [==============================] - 1s 30ms/step - loss: 4.9879
Epoch 35/50
22/22 [==============================] - ETA: 0s - loss: 4.7025
Epoch 35: loss did not improve from 4.40191
22/22 [==============================] - 1s 28ms/step - loss: 4.7025
Epoch 36/50
22/22 [==============================] - ETA: 0s - loss: 7.2521
Epoch 36: loss did not improve from 4.40191
22/22 [==============================] - 1s 27ms/step - loss: 7.2521
Epoch 37/50
21/22 [===========================>..] - ETA: 0s - loss: 5.4934
Epoch 37: loss did not improve from 4.40191
22/22 [==============================] - 1s 31ms/step - loss: 5.4658
Epoch 38/50
21/22 [===========================>..] - ET

Epoch 24/50
20/22 [==========================>...] - ETA: 0s - loss: 25.2745
Epoch 24: loss did not improve from 24.98680
22/22 [==============================] - 1s 26ms/step - loss: 25.3073
Epoch 25/50
21/22 [===========================>..] - ETA: 0s - loss: 25.2717
Epoch 25: loss did not improve from 24.98680
22/22 [==============================] - 1s 28ms/step - loss: 25.3784
Epoch 26/50
21/22 [===========================>..] - ETA: 0s - loss: 23.1350
Epoch 26: loss improved from 24.98680 to 23.26049, saving model to ./EEMD_saved_models\WTI_residual_best_model.h5
22/22 [==============================] - 1s 33ms/step - loss: 23.2605
Epoch 27/50
21/22 [===========================>..] - ETA: 0s - loss: 33.2620
Epoch 27: loss did not improve from 23.26049
22/22 [==============================] - 1s 31ms/step - loss: 33.2038
Epoch 28/50
21/22 [===========================>..] - ETA: 0s - loss: 30.1483
Epoch 28: loss did not improve from 23.26049
22/22 [==============================] - 

In [41]:
# step 4 Prediction/Decomposition

In [53]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Load trained models
imf1_model = load_model('./EEMD_saved_models/WTI_imf1_best_model.h5')
imf2_model = load_model('./EEMD_saved_models/WTI_imf2_best_model.h5')

# Load data
data = pd.read_excel('./Datasets/CEEMDAN Decomposed Data.xlsx')
data.sort_values('time_idx', inplace=True)

# Define prediction range
start_time_idx = 713 
end_time_idx = data['time_idx'].max() 

# Predict and update DataFrame
for time_idx in range(start_time_idx, end_time_idx + 1):
    target_idx = time_idx - 1

    if target_idx >= 0:
        window_data = data.iloc[time_idx - 30:time_idx][['WTI', 'time_idx']].values.flatten()
        window_data = np.expand_dims(window_data, axis=0)

        # Predict
        predicted_imf1 = imf1_model.predict(window_data)  
        predicted_imf2 = imf2_model.predict(window_data) 

        # Update DataFrame
        data.at[target_idx, 'WTI_IMF1'] = predicted_imf1.flatten()[0]
        data.at[target_idx, 'WTI_IMF2'] = predicted_imf2.flatten()[0]

        # Calculate and adjust residual
        if 'WTI' in data.columns:
            predicted_residual = data.at[target_idx, 'WTI'] - (predicted_imf1.flatten()[0] + predicted_imf2.flatten()[0])
            data.at[target_idx, 'WTI_Res'] = predicted_residual + 20  # Include +20 adjustment here

# Save the updated DataFrame back to Excel
data.to_excel('./Datasets/CEEMDAN Decomposed Data.xlsx', index=False)

print("Predictions integrated and Excel file updated.")


1/1 [==============================] - 0s 29ms/step
Predictions integrated and Excel file updated.
